In [1]:
# THERE ARE SEPARATE NOTEBOOKS FOR VISUALIZATIONS, DATASET ANALYSIS, ETC. IN THE REPO.

import pandas as pd
import numpy as np

# READ THE CSV INTO DATAFRAME

df = pd.read_csv('Syngenta/Syngenta_2017/Experiment_dataset.csv')

In [ ]:
# THIS IS USED WHEN CONNECTING TO COMPUTE NODES TO CHECK IF EVERYTHING IS WORKING

import os

print(os.getcwd())

In [ ]:
# CURRENTLY NECESSARY IF: USING 174 ADDITIONAL VARIETY COLUMNS METHOD

variety_dummies = pd.get_dummies(df.Variety)
df = pd.concat([df, variety_dummies], axis=1)

In [ ]:
# CURRENTLY NECESSARY IF: USING 8 ADDITIONAL VARIETY COLUMNS (BINARY REPRESENTATION)

UNIQUE_VARIETIES = np.unique(df.Variety)
INDICES = [i for i in range(0, len(UNIQUE_VARIETIES))]
# print(np.where(UNIQUE_VARIETIES=='V000017'))
df.Variety = df.Variety.apply(lambda var: np.where(UNIQUE_VARIETIES==var)[0][0])
# print(UNIQUE_VARIETIES)
# print(df.Variety)

df.Variety = df.Variety.apply(lambda var: '{0:08b}'.format(var))
print("done")
df.Variety = df.Variety.apply(lambda var: list(var))
print("done")
variety_binary_df = pd.DataFrame(pd.Series(df.Variety[i]) for i in range(0, len(df.Variety)))
df = pd.concat([df, variety_binary_df], axis=1)

In [2]:
# GOAL OF THIS MODULE:
# Encode the planting date as a season

# remove the dates that are "."
df = df[~df['Planting date'].str.match("\.")]
plant_date = df['Planting date'].apply(lambda dt: pd.to_datetime(dt))
plant_months = plant_date.apply(lambda dt: dt.month)
season = plant_date.rename("Season")
season = pd.to_datetime(season)
season = season.apply(lambda dt: (dt.month%12 + 3)//3)
# df['Plant date'] = pd.to_datetime(df['Plant date'])
df = pd.concat([df, season], axis=1)

# plant_date = pd.to_datetime(df['Planting date'], infer_datetime_format=True)
# df = df['Planting date'].apply(lambda dt: (dt.month%12 + 3)//3)
# pd.get_dummies(df['Planting date'])

In [3]:
# ADD MONTH OF MAY AND JUNE ONE HOT ENCODING INTO THE DATAFRAME
pd.get_dummies(plant_months).sum()
june = pd.get_dummies(plant_months).loc[:,6]
june = june.rename("June")
may = pd.get_dummies(plant_months).loc[:,5]
may = may.rename("May")
df = pd.concat([df, may], axis=1)
df = pd.concat([df, june], axis=1)

In [4]:
# LATITUDE AND LONGITUDE CLUSTERING INTO FEATURES

from sklearn.cluster import KMeans

latlong = df.loc[:, ['Latitude', 'Longitude']]

kmeans = KMeans(n_clusters=4, random_state=0, n_jobs=-1).fit(latlong)
kmeans.labels_.shape
lat_long_dummies = pd.get_dummies(kmeans.labels_)
lat_long_dummies = lat_long_dummies.rename(index=int, columns={0: "Loc Clust 0",
                                                               1: "Loc Clust 1",
                                                               2: "Loc Clust 2",
                                                               3: "Loc Clust 3"})
df = pd.concat([df, lat_long_dummies], axis = 1)

In [5]:
#REMOVE ANY NAN VALUES

print(df.columns)
df = df[~df.Silt.isnull()]
df = df[~df['Loc Clust 1'].isnull()]

Index(['Experiment', 'Location', 'Variety', 'Planting date', 'Yield',
       'Check Yield', 'Yield difference', 'Year', 'Latitude', 'Longitude',
       'Temperature', 'Precipitation', 'Solar Radiation', 'Soil class', 'CEC',
       'Organic matter', 'pH', 'Clay', 'Silt', 'Sand', 'PI', 'Area', 'Season',
       'May', 'June', 'Loc Clust 0', 'Loc Clust 1', 'Loc Clust 2',
       'Loc Clust 3'],
      dtype='object')


In [ ]:
for col in df.columns:
    print(col, type(df[col][0]))

In [6]:
# DROP ALL THE CELLS THAT ARE NOT USABLE SUCH AS THE ONES THAT ARE STRINGS OR DATES

# set if want to drop some columns specifically
should_drop = 1
# columns_to_drop = ['Experiment', 'Location',
#                    'Check Yield', 'Yield difference', 'Latitude',
#                    'Longitude', 'PI', 'Variety', 'Planting date', 'Season']

# BELOW DROP IS USED FOR THE DF_DICT APPROACH
columns_to_drop = ['Experiment', 'Location',
                   'Check Yield', 'Yield difference', 'Latitude',
                   'Longitude', 'PI', 'Planting date', 'Season']

# set if want to keep some columns specifically
should_keep = 0
# columns_to_keep = ['Loc Clust 0', 'Loc Clust 1', 'Loc Clust 2', 'Loc Clust 3']
columns_to_keep_top = ['Silt', 'Precipitation', 'Temperature', 'Solar Radiation', 'Organic matter']
# columns_VARIETIES_ONLY = np.asarray(df.iloc[:, df.columns.str.match('V\d\d\d\d\d\d')].columns)

#set the below variable to whatever columns you want to keep
columns_to_keep = columns_to_keep_top

MUST_HAVE_COLUMNS = ['Yield']
# print(columns_to_keep)

df = df.drop(columns_to_drop, axis=1) if should_drop else df
df = df.loc[:, np.concatenate((columns_to_keep, MUST_HAVE_COLUMNS))] if should_keep else df
df['YieldBucket'] = pd.Series(pd.qcut(df.Yield, q=3, labels=["high", "medium", "low"]))
print("The final dataframe has columns: ", df.columns)

The final dataframe has columns:  Index(['Variety', 'Yield', 'Year', 'Temperature', 'Precipitation',
       'Solar Radiation', 'Soil class', 'CEC', 'Organic matter', 'pH', 'Clay',
       'Silt', 'Sand', 'Area', 'May', 'June', 'Loc Clust 0', 'Loc Clust 1',
       'Loc Clust 2', 'Loc Clust 3', 'YieldBucket'],
      dtype='object')


In [ ]:
# LET US ALSO MAKE SURE THERE ARE NO NAN IN THE DATA

print("We expect to be %s nan values and there actually are %s nan values\n" % (0, np.sum(df.isnull().sum())))
print(df.isnull().sum())
# AFTER COLUMNS, MAKE SURE NO SKETCHY ONES
for col in df.columns:
    print(col, type(df[col][0]))    

In [8]:
# GOAL OF THIS MODULE:
# CREATE A DICTIONARY OF DATAFRAMES CONTAINING EACH VARIETY AND CORRESPONDING ROWS
# MOST LIKELY SHOULD DO THIS AFTER ADDING AND REMOVING THE DESIRED COLUMNS

UNIQUE_VARIETIES = np.unique(df.Variety)
df_dict = {variety: df.loc[df.Variety == variety] for variety in UNIQUE_VARIETIES}


In [108]:
df_dict['V000016']

,Variety,Yield,Year,Temperature,Precipitation,Solar Radiation,Soil class,CEC,Organic matter,pH,...,Silt,Sand,Area,May,June,Loc Clust 0,Loc Clust 1,Loc Clust 2,Loc Clust 3,YieldBucket
95,V000016,77.57,2009.0,3573.1,569.0,1041432.0,212.0,25.623264,4.576477,6.242766,...,51.771195,22.923973,0.4143,1.0,0.0,1.0,0.0,0.0,0.0,low
98,V000016,68.30,2009.0,3466.3,523.1,1028812.0,213.0,22.239511,4.782916,6.206655,...,40.129485,39.627604,0.3422,1.0,0.0,1.0,0.0,0.0,0.0,low
102,V000016,74.64,2009.0,3573.1,569.0,1041432.0,212.0,25.623264,4.576477,6.242766,...,51.771195,22.923973,0.4143,1.0,0.0,1.0,0.0,0.0,0.0,low
105,V000016,61.31,2009.0,3466.3,523.1,1028812.0,213.0,22.239511,4.782916,6.206655,...,40.129485,39.627604,0.3422,1.0,0.0,1.0,0.0,0.0,0.0,medium
107,V000016,72.73,2009.0,3573.1,569.0,1041432.0,212.0,25.623264,4.576477,6.242766,...,51.771195,22.923973,0.4143,1.0,0.0,1.0,0.0,0.0,0.0,low
109,V000016,67.76,2009.0,3466.3,523.1,1028812.0,213.0,22.239511,4.782916,6.206655,...,40.129485,39.627604,0.3422,1.0,0.0,1.0,0.0,0.0,0.0,low
111,V000016,64.54,2009.0,3466.3,523.1,1028812.0,213.0,22.239511,4.782916,6.206655,...,40.129485,39.627604,0.3422,1.0,0.0,1.0,0.0,0.0,0.0,low
114,V000016,65.61,2009.0,3466.3,523.1,1028812.0,213.0,22.239511,4.782916,6.206655,...,40.129485,39.627604,0.3422,1.0,0.0,1.0,0.0,0.0,0.0,low
117,V000016,59.16,2009.0,3466.3,523.1,1028812.0,213.0,22.239511,4.782916,6.206655,...,40.129485,39.627604,0.3422,1.0,0.0,1.0,0.0,0.0,0.0,medium
120,V000016,57.54,2009.0,3466.3,523.1,1028812.0,213.0,22.239511,4.782916,6.206655,...,40.129485,39.627604,0.3422,1.0,0.0,1.0,0.0,0.0,0.0,medium


In [26]:
from sklearn.model_selection import train_test_split

def cross_validation_split(data):
    
    
    if type(data) == dict:
        X_train_dict = {}
        X_test_dict = {}
        y_train_dict = {}
        y_test_dict = {}
        for variety, dataf in data.items():
            X = dataf.drop(['Yield', 'YieldBucket', 'Variety'], axis=1)
            y = dataf.Yield
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.05, train_size = 0.95,
                                                                random_state = 42)
            
            X_train_dict[variety] = X_train
            X_test_dict[variety] = X_test
            y_train_dict[variety] = y_train
            y_test_dict[variety] = y_test
            
        return X_train_dict, X_test_dict, y_train_dict, y_test_dict
#     else:
#         # ADD BELOW MODULE HERE
            
            
X_train_dict, X_test_dict, y_train_dict, y_test_dict = cross_validation_split(df_dict)

In [111]:
y_train_dict['V000016']

5348     74.39
461      51.24
109      67.76
752      74.84
5483     76.79
532      54.93
12561    51.86
810      69.81
14327    69.80
3561     76.12
4397     62.08
612      48.38
13972    55.85
12449    56.98
5901     53.78
12489    59.12
5477     43.56
12988    57.08
5467     74.28
565      66.38
485      77.53
5423     69.25
5442     65.84
12617    58.88
521      75.30
4401     46.92
5460     70.03
530      61.36
13960    67.97
758      69.25
         ...  
5463     54.97
5632     73.16
21884    53.50
5624     41.59
5450     43.71
583      76.51
539      64.78
9606     38.90
12481    62.19
172      69.37
5412     90.70
12577    52.22
5640     47.09
12645    50.82
12501    58.08
13968    65.40
5368     72.38
5292     68.13
5376     69.47
829      43.00
788      58.08
8893     44.37
5256     74.13
21894    55.64
168      60.32
5628     41.10
709      48.03
3505     61.19
12553    57.70
840      45.80
Name: Yield, Length: 310, dtype: float64

In [ ]:
# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT
# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT

X = df.drop(['Yield', 'YieldBucket'], axis=1)

# print(X.columns)

y = df.Yield

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.05, train_size = 0.95, random_state = 42)

INPUT_COLS = X_train.columns
# TEST_COLS = y_train.columns

In [ ]:
print("X_train shape:", X_train.shape, "\ny_train shape:", y_train.shape)

In [ ]:
for val in X_train.columns:
    print(val)

In [ ]:
# GOAL OF THIS MODULE: PCA OUTPUT AND VISUAL ANALYSIS

from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

def gen_PCA(X_train_, target_series, visualize=False):
    # vis_features = ['Silt', 'Precipitation', 'Organic matter', 'Solar Radiation', 'Temperature', 'CEC']
    # X_train_scaled = StandardScaler().fit_transform(X_train_.loc[:, vis_features])
    X_train_scaled = StandardScaler().fit_transform(X_train_)

    pca = PCA(n_components=8)
    principalComponents = pca.fit_transform(X_train_scaled)
#     principal_df = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2'])
    principal_df = pd.DataFrame(data=principalComponents)
    print(principal_df.shape)
#     print(X_train_.shape)
    print(target_series.shape)
    
    # WHY DOES THE BELOW LINE NOT WORK AND INSTEAD I NEED TO USE THE FOLLOWING TWO LINES?
#     final_df = pd.concat([principal_df, target_series], axis=1)
    final_df = principal_df
    final_df['Yield'] = target_series
    
    print(final_df.shape)
    print(final_df)
    if visualize:
        fig = plt.figure(figsize=(20,20))
        ax = fig.add_subplot(1,1,1) 
        ax.set_xlabel('Principal Component 1', fontsize = 15)
        ax.set_ylabel('Principal Component 2', fontsize = 15)
        ax.set_title('2 component PCA', fontsize = 20)
        targets = ['high', 'medium', 'low']
        colors = ['g', 'y', 'r']
        for target, color in zip(targets,colors):
            indicesToKeep = final_df['YieldBucket'] == target
            ax.scatter(final_df.loc[indicesToKeep, 'principal component 1']
                       , final_df.loc[indicesToKeep, 'principal component 2']
                       , c = color
                       , s = 3)
        ax.legend(targets)
        ax.grid()
        fig.savefig("graphs/PCA-bucketing.png")
    return final_df
    
final_df = gen_PCA(X_train, df['Yield'])

In [ ]:
len(X_train.columns)
final_df.isnull().sum()
final_df[final_df.Yield.isnull()]

In [ ]:
final_df = final_df[~final_df.Yield.isnull()]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(final_df.drop(['Yield'], axis=1), final_df.loc[:, 'Yield'], train_size = 0.5, test_size = 0.1)

In [ ]:
for i in range(0,8):
    df[i] = pd.to_numeric(df[i])

In [34]:
# THIS FUNCTION WILL EVALUATE ERRORS BASED ON RMSE (FROM SYNGENTA CHALLENGE SPEC)
# AND ALSO WILL EVALUATE BASED ON AVERAGE ERROR

from sklearn.metrics import mean_squared_error

def evaluate_errors(prediction, actual):
    RMSE_error = np.sqrt(mean_squared_error(prediction, actual))
    print("RMSE Error: ", np.sqrt(mean_squared_error(prediction, actual)))
    avg_error_vector = np.absolute(((prediction - actual) / actual) * 100)
    print("Average Error details:\n", avg_error_vector.describe())
    return avg_error_vector, RMSE_error

In [ ]:
# MOST USED CLASSIFIER CURRENTLY

from sklearn.ensemble import RandomForestRegressor

regr = RandomForestRegressor(n_estimators=20, max_depth=13, random_state=0, verbose=1, n_jobs=-1)
regr.fit(X_train, y_train)
preds = regr.predict(X_test)
evaluate_errors(preds, y_test)

In [ ]:
# GET OUTPUT OF FEATURE IMPORTANCE

def get_feature_importances(regr):
    feature_importances = regr.feature_importances_
    feature_importances = pd.Series(feature_importances)
    feature_importance_df = pd.DataFrame({'feature': X_train.columns,'feature_importance': feature_importances})
    feature_importance_df = feature_importance_df.sort_values(by=['feature_importance'])
    for index, row in feature_importance_df.iterrows():
        print(row['feature'], 'has importance: ', row['feature_importance'])
get_feature_importances(regr)

In [ ]:
y_train_dict['V000016']

In [ ]:
# THIS IS TO BE USED WITH THE DICTIONARY APPROACH ONLY

from sklearn.base import clone

def train_on_varieties(clf, X_train_dict_, y_train_dict_):
    clfs = {}
    for variety, dataf in X_train_dict_.items():
        clf = clone(clf)
        clf.fit(X_train_dict_[variety], y_train_dict[variety])
        clfs[variety] = clf
    return clfs

from sklearn.ensemble import RandomForestRegressor

clf = RandomForestRegressor(n_estimators=20, max_depth=13, random_state=0, verbose=1, n_jobs=-1)

clfs = train_on_varieties(clf, X_train_dict, y_train_dict)

In [112]:
clfs

{'V000016': RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=13,
            max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=-1,
            oob_score=False, random_state=0, verbose=1, warm_start=False),
 'V000017': RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=13,
            max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=-1,
            oob_score=False, random_state=0, verbose=1, warm_start=False),
 'V000018': RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=13,
            max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
       

In [35]:
def test_on_varieties(classifier_dict, X_test_dict_, y_test_dict_):
    preds = {}
    accuracies = {}
    for variety, classifier in classifier_dict.items():
        preds[variety] = classifier.predict(X_test_dict_[variety])
        accuracies[variety] = evaluate_errors(preds[variety], y_test_dict_[variety])
    return preds, accuracies
        
preds, accuracies = test_on_varieties(clfs, X_test_dict, y_test_dict)

[Parallel(n_jobs=4)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done  20 out of  20 | elapsed:    0.0s finished


RMSE Error:  9.723769578091776
Average Error details:
 count    16.000000
mean     12.262889
std       7.951827
min       2.151085
25%       5.605886
50%      13.592675
75%      16.638164
max      29.146325
Name: Yield, dtype: float64
RMSE Error:  7.396315563621307
Average Error details:
 count     6.000000
mean      6.737508
std       8.066563
min       0.937229
25%       2.817232
50%       3.577429
75%       6.195710
max      22.700658
Name: Yield, dtype: float64


[Parallel(n_jobs=4)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done  20 out of  20 | elapsed:    0.0s finished


RMSE Error:  7.792062372364676
Average Error details:
 count     7.000000
mean     10.042122
std      11.088144
min       1.510542
25%       4.263739
50%       4.576724
75%      10.968636
max      33.742835
Name: Yield, dtype: float64
RMSE Error:  6.987104450395963
Average Error details:
 count     7.000000
mean      9.534390
std       8.307492
min       0.228011
25%       2.695396
50%       8.661244
75%      15.490838
max      21.479009
Name: Yield, dtype: float64


[Parallel(n_jobs=4)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done  20 out of  20 | elapsed:    0.0s finished


...

RMSE Error:  7.102117962861548
Average Error details:
 count     9.000000
mean      9.466350
std       6.024396
min       2.600802
25%       5.884170
50%       7.622622
75%      12.464270
max      20.700062
Name: Yield, dtype: float64
RMSE Error:  9.502769990902648
Average Error details:
 count     6.000000
mean     12.018114
std      17.439586
min       0.228436
25%       1.799974
50%       6.936799
75%      10.261559
max      46.549141
Name: Yield, dtype: float64


[Parallel(n_jobs=4)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done  20 out of  20 | elapsed:    0.0s finished


RMSE Error:  7.08068047971655
Average Error details:
 count    29.000000
mean      9.981562
std      11.884690
min       0.487976
25%       2.058295
50%       7.117205
75%       8.572177
max      46.641285
Name: Yield, dtype: float64
RMSE Error:  5.877714366305578
Average Error details:
 count     8.000000
mean      8.228587
std       5.852452
min       0.585526
25%       5.196139
50%       7.064792
75%      10.463074
max      20.133307
Name: Yield, dtype: float64


[Parallel(n_jobs=4)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done  20 out of  20 | elapsed:    0.0s finished


RMSE Error:  5.0236996451860065
Average Error details:
 count     5.000000
mean      5.917181
std       6.425372
min       0.260439
25%       1.084329
50%       2.433140
75%      12.812754
max      12.995242
Name: Yield, dtype: float64
RMSE Error:  10.329422907051253
Average Error details:
 count    10.000000
mean     13.205203
std      12.036093
min       1.470773
25%       3.476676
50%       8.391991
75%      22.299311
max      36.360896
Name: Yield, dtype: float64


In [115]:
# THIS IS SOME ANALYSIS OF THE PREDICTIONS OF THE DICTIONARY APPROACH

# for var, accs in accuracies.items():
#     print(accs)
# #     print(var, accs[0].describe())
#     all_means = np.array([])
#     np.append(all_means, accs[0].describe()['mean'])
# #     print(accs[0].describe()['mean'])
#     print(all_means.shape)
all_means = np.array([])
all_RMSE = np.array([])

for variety, accuracy_tuple in accuracies.items():
    all_means = np.append(all_means, accuracy_tuple[0].describe()['mean'])
    all_RMSE = np.append(all_RMSE, accuracy_tuple[1])
# accuracies['V000016'][0].describe()['mean']
# print(all_means.shape)
import pprint
from scipy.stats import describe
pprint.pprint(describe(all_means))
np.mean(all_means)
# print(all_means)
print(np.mean(all_RMSE))

DescribeResult(nobs=174, minmax=(3.3428277395416277, 40.93930403334446), mean=11.168134003433787, variance=23.875102467369445, skewness=2.6029817425546242, kurtosis=10.929261165686627)
7.656917971214786


In [ ]:
# CREATE AN ARRAY OF POTENTIAL CLASSIFIERS TO TEST THE PERFORMANCE OF EACH AND COMPARE

from sklearn import linear_model
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import ensemble

GRAD_BOOST_PARAMS = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}

classifiers = [
#     MLPRegressor(solver='lbfgs', alpha=1e-5,
#                      hidden_layer_sizes=(5, 2), random_state=1),
#     RandomForestRegressor(n_estimators=20, max_depth=13, random_state=0, verbose=1, n_jobs=-1),
#     linear_model.SGDRegressor(),
#     linear_model.BayesianRidge(),
#     linear_model.LassoLars(),
#     linear_model.ARDRegression(),
#     linear_model.PassiveAggressiveRegressor(),
#     linear_model.TheilSenRegressor(),
#     linear_model.LinearRegression(),
    LGBMRegressor(n_estimators=1000, learning_rate=0.01),
    svm.SVR(kernel="linear"),
    linear_model.ARDRegression(),
    ensemble.GradientBoostingRegressor(**GRAD_BOOST_PARAMS)]

In [ ]:
# THIS MODULE WILL OUTPUT A DICTIONARY WITH THE FEATURE RANKING OF A CLASSIFIER

from sklearn.feature_selection import RFECV
def RFEVC_(clf, X_train_, y_train_, step_=1, verbose_=0):
    selector = RFECV(clf, step=step_, verbose=verbose_, n_jobs=-1)
    selector = selector.fit(X_train_, y_train_)
    
    INPUT_COLS = X_train.columns
    RANKS = selector.ranking_

    rank_dict = {}
    for idx, item in enumerate(RANKS):
        rank_dict[INPUT_COLS[idx]] = item
    
    import pprint
    pprint.pprint(rank_dict)
    
    return rank_dict, selector

# USAGE EXAMPLE:
# rank_dict, selec = RFEVC_(regr, X_train, y_train)

In [ ]:
# RUN THE RECURSIVE FEATURE ELIMINATION ON EACH CLASSIFIER IN THE ARRAY

for item in classifiers:
    print(item)
    clf = item
    rank_dict, selector = RFEVC_(clf, X_train, y_train)

In [ ]:
# RUN THE CLASSIFIERS AND PREDICT ON TEST DATA

for item in classifiers:
    print(item)
    clf = item
    clf.fit(X_train, y_train)
    preds = clf.predict(X_test)
    errors = evaluate_errors(preds, y_test)
    try:
        get_feature_importances(clf)
    except:
        print("NO FEATURE IMPORTANCE METRIC")

In [ ]:
import random

NUM_VARIETIES = 174

def best_yield_variety(regr, test_set, random_sel = True, n_samples = 174, print_variety_preds = True):
    
    #create empty df
    dup_df = pd.DataFrame()
    
    #choose a rand sample of input test_set (for dev purposes, wouldn't be used in app)
    test_set_sample = test_set.sample(n= n_samples) if random_sel else test_set
    
    #progress of intensive, long for loop upcoming
    counter = 0
    
    #for loop will, for each row in test_set_sample, duplicate that row by NUM_VARIETIES
    for index, row in test_set_sample.iterrows():
        counter+=1
        print(counter)
        dup_df = dup_df.append([row] * NUM_VARIETIES, ignore_index = True)
        
    #extract the varieties columns
    duplicated_df_varieties = dup_df.loc[:, dup_df.columns.str.match('V\d\d\d\d\d\d')]
    #extract the names of the varieties
    varieties_array = duplicated_df_varieties.columns
    num_expanded_data_pts = duplicated_df_varieties.shape[0]
    #we must have a zeroed matrix of the same shape as the duplicated_df_varieties
    #so that we can input a 1 just once for each variety per 174 rows
    d = np.zeros((duplicated_df_varieties.shape[0], NUM_VARIETIES))
    #make d our dataframe, with columns equal to the varieties
    duplicated_df_varieties = pd.DataFrame(d, columns=varieties_array)
    #for loop will place a 1 just once for each variety per 174 rows (one hot rep)
    for i in range(duplicated_df_varieties.shape[0]):
        var_index = i % 174
        duplicated_df_varieties.loc[i, varieties_array[var_index]] = 1
    #remove the varieties (will be added back with the new values)
    dup_df = dup_df.drop(varieties_array, axis = 1)
    #add the new values (one hot representations) from above for loop
    dup_df = pd.concat([dup_df, duplicated_df_varieties], axis=1)
    #do prediction on the entire dataframe
    preds_per_variety = regr.predict(dup_df)
    #*******make it into a dataframe where each row will give the performance of each variety
    #with the same environmental conditions*******
    preds_df = pd.DataFrame(preds_per_variety.reshape((int(num_expanded_data_pts/NUM_VARIETIES), NUM_VARIETIES)),
                            columns=varieties_array)
    
    #environmental conditions (everything except the variety data)
    envcond = test_set_sample.drop(varieties_array, axis=1)
    
    #a simple print out for best variety given the environmental conditions
    hr_preds = []
    if print_variety_preds:
        envcond_cols = envcond.columns
        counter = -1
        for idx, row in envcond.iterrows():
            counter+=1
            out = "For environmental conditions:\n%s\nthe best variety is:%s" % (row, preds_df.idxmax(axis=1)[counter])
            hr_preds.append(out)
            print(out)
    
    return preds_df, envcond, hr_preds
    
        
preds_df, envcond, hr_preds = best_yield_variety(regr, X_test, n_samples = 500)

In [ ]:
#find number of times a variety is the maximum in the above prediction dataframe
maxes = preds_df.idxmax(axis=1) # get first max variety per env
# print(pd.get_dummies(maxes).describe())
print(pd.get_dummies(maxes).sum().sort_values(ascending=False))

In [ ]:
#find number of times a variety is the maximum in the above prediction dataframe
maxes = preds_df.idxmax(axis=1) # get first max variety per env
# print(pd.get_dummies(maxes).describe())
print(pd.get_dummies(maxes).sum().sort_values(ascending=False))
print(len(pd.get_dummies(maxes).sum()))

In [ ]:
preds_df.describe().sort_values(by="mean", axis=1)

In [ ]:
preds_df.describe().loc['mean',:].sort_values().describe()

In [ ]:
# CLASSIFICATION METHODS BELOW!

In [ ]:
# THIS WILL ONLY WORK WITH THE BUCKET METHOD

from sklearn.ensemble import RandomForestClassifier
regr = RandomForestClassifier(n_estimators=10, max_depth=20, random_state=0, verbose=1)
regr.fit(X_train, y_train)
preds = regr.predict(X_test)

from sklearn.metrics import accuracy_score

print(accuracy_score(y_test, preds))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

classifiers = [
#     KNeighborsClassifier(3),
#     SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
#     GaussianProcessClassifier(1.0 * RBF(1.0)),
#     DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]
from sklearn.metrics import accuracy_score
for item in classifiers:
    print(item)
    clf = item
    clf.fit(scale(X_train), y_train)
    preds = clf.predict(scale(X_test))
    print(accuracy_score(y_test, preds))
#     errors = np.absolute(((preds - y_test) / y_test) * 100)
#     print(errors)
#     print(np.mean(errors))